<h1><center>Limpieza Properatti</center></h1>

#### CN07
#### Alejandro Susmelj


In [1]:
import numpy as np
import pandas as pd
import re
from scipy import stats
from math import floor
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

In [7]:
#Leo el dataset y lo asigno a una variable

dataset_original = pd.read_csv("../data/properatti.csv", sep = ",")
dataset_original

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121215,121215,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,NaN,NaN,...,93.0,7699.115044,9354.838710,NaN,NaN,10000.0,http://www.properati.com.ar/1cja2_venta_depart...,TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...,Torre Forum Alcorta- Impecable 3 ambientes,https://thumbs4.properati.com/1/bjms0KnaAnlNoQ...
121216,121216,sell,house,Beccar,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,Argentina,Bs.As. G.B.A. Zona Norte,3436080.0,NaN,NaN,...,360.0,1383.333333,1383.333333,NaN,NaN,NaN,http://www.properati.com.ar/1cja6_venta_casa_b...,Excelente e impecable casa en Venta en Las Lom...,Ruca Inmuebles | Venta | Lomas de San Isidro |...,https://thumbs4.properati.com/2/PCc3WuQDjpNZc4...
121217,121217,sell,apartment,Villa Urquiza,|Argentina|Capital Federal|Villa Urquiza|,Argentina,Capital Federal,3433775.0,"-34.5706388726,-58.4755963355",-34.570639,...,39.0,2858.695652,3371.794872,NaN,NaN,NaN,http://www.properati.com.ar/1cja7_venta_depart...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,https://thumbs4.properati.com/9/YAe_-2gRVykADP...
121218,121218,sell,apartment,Plaza Colón,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,NaN,NaN,NaN,...,48.0,1997.916667,1997.916667,NaN,NaN,NaN,http://www.properati.com.ar/1cja8_venta_depart...,"2 Amb al contrafrente, luminoso. El departame...",2 amb. C/ dep. de servicio al contrafrente| Re...,https://thumbs4.properati.com/8/Q12PTvU6BQJ0ib...


In [3]:
print("Tamaño del dataset:", dataset_original.shape)

Tamaño del dataset: (121220, 26)


# Análisis global

Primero, elimino variables innecesarias para obtener mayor claridad a la hora de analizar / limpiar el dataset

#### Unnamed 0

In [4]:
# La variable Unnamed: 0, es un índice repetido. Por lo que, la elimino.

dataset = dataset_original.drop("Unnamed: 0", axis = 1)

#### Operation

In [5]:
# Muestro la cantidad de filas en la columna Operation que sean distintas a "sell"

dataset_original.loc[dataset_original["operation"] != "sell", "operation"].sum()

0

In [6]:
#Ya que todas las filas contienen el dato "sell", elimino la variable Operation

dataset.drop("operation", axis = 1, inplace = True)

#### Country name

In [7]:
# Verifico si existe alguna fila de country name distinta al dato "Argentina" o que sea nula.

dataset_original.country_name.loc[(dataset_original.country_name != "Argentina") | (dataset_original.country_name.isnull()) ].sum()

0

In [8]:
# Elimino la variable country name

dataset.drop("country_name", axis = 1, inplace = True)

#### Lat - Lon

Las variables Lat y Lon se encuentran contenidas dentro de la variable lat-lon por lo que, la variable agrupadora Lat-Lon podría ser eliminada

In [9]:
# Analizo si alguna fila de Lat o Lon tiene Nans que puedan ser completados con Lat-Lon

dataset_original.loc[(dataset_original["lat-lon"].notnull()) & ((dataset_original["lat"].isnull()) | (dataset_original["lon"].isnull())), ["lat-lon", "lat", "lon"]]

,lat-lon,lat,lon


In [10]:
# Elimino la variable lat-lon

dataset.drop("lat-lon", axis = 1, inplace = True)


#### Place with parent names

Esta variable es una acumulación de otras 3 columnas por lo cual, la descarto ya que prefiero las variables individuales

Antes de eliminarla, me percato de que tengo nans en place name que pueden ser rellenados con información obtenida de esta variable

In [11]:
# Muestro los Nan alojados en la variable place name

dataset.loc[dataset.place_name.isnull(), ["place_name", "place_with_parent_names"]]

,place_name,place_with_parent_names
6489,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||
10201,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||
11451,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||
14839,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||
18622,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||
21922,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||
23664,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||
24722,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||
38856,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||
45970,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||


Gracias a la información brindada por la columna place with parent names, puedo imputar el dato "Tigre" en todos los Nan de place name

In [12]:
dataset.place_name.fillna("Tigre", inplace = True)

Debido a que las variables place name y state name no tienen ningun Nan, elimino a la variable place with parent names la cual, contiene a ambas

In [13]:
dataset.drop("place_with_parent_names", axis = 1, inplace = True)

#### Price y Currency

Cuando la variable currency:

   * Caso 1: Tiene como dato "ARS", la variable price se convierte en un redondedo de la variable price aprox local currency
    
   * Caso 2: Tiene como dato "USD", la variable price es igual a price aprox usd
    
   * Caso 3: Tiene como dato otro tipo de moneda, asumo que la conversión a "ARS" y "USD" es la correcta


In [14]:
# Caso 1:

# Muestro por pantalla los registros donde la moneda sea "ARS"

dataset_original.loc[(dataset_original.currency == "ARS"), ["price", "currency", "price_aprox_local_currency", "price_aprox_usd"]].head(5)

,price,currency,price_aprox_local_currency,price_aprox_usd
9,460000.0,ARS,455201.45,25798.49
32,550000.0,ARS,544262.59,30846.02
37,1250000.0,ARS,1236960.43,70104.59
78,5500000.0,ARS,5442626.35,308460.22
97,1423800.0,ARS,1408947.37,79851.93


In [15]:
# Caso 2:

# Muestro por pantalla los registros donde la moneda sea "USD"

dataset_original.loc[(dataset_original.currency == "USD"), ["price", "currency", "price_aprox_local_currency", "price_aprox_usd"]].head(5)

,price,currency,price_aprox_local_currency,price_aprox_usd
0,62000.0,USD,1093959.0,62000.0
1,150000.0,USD,2646675.0,150000.0
2,72000.0,USD,1270404.0,72000.0
3,95000.0,USD,1676227.5,95000.0
4,64000.0,USD,1129248.0,64000.0


In [16]:
# Caso 3:

# Muestro por pantalla los registros donde la moneda no sea "ARS" ni "USD". Además, solo muestro los registros que no sean nulos.

dataset_original.loc[(dataset_original.currency != "ARS") & (dataset_original.currency != "USD") & (dataset_original.currency.notnull()), ["price", "currency", "price_aprox_local_currency", "price_aprox_usd"]]

,price,currency,price_aprox_local_currency,price_aprox_usd
50387,380000.0,PEN,2.066865e+06,117139.33
50388,950000.0,PEN,5.167162e+06,292848.33
107390,650000000.0,UYU,4.054773e+08,22980378.29


In [17]:
# Analizo si la variable price me puede servir para llenar un campo de price aprox local currency o price aprox usd que este vacio

dataset_original.loc[(dataset_original.price.notnull()) & (dataset_original.price_aprox_local_currency.isnull() | dataset_original.price_aprox_usd.isnull()), ["price", "currency", "price_aprox_local_currency", "price_aprox_usd"]]

,price,currency,price_aprox_local_currency,price_aprox_usd


Teniendo en cuenta que las monedas más utilizadas en el dataset son "ARS" y "USD", me quedo únicamente con las variables price aprox local currency y price aprox usd.

In [18]:
# Descarto price y currency

dataset.drop("price", axis = 1, inplace = True)
dataset.drop("currency", axis = 1, inplace = True)

#### Price aprox local currency

Antes de eliminar esta variable, la utilizo para calcular el valor del dólar

Conversión: Precio en moneda local / Precio en dólares = Valor del dólar

In [19]:
# Creo un dataset con las filas que no tengan nulos ni ceros en price aprox usd

dataset_dolar = dataset.loc[(dataset_original.price_aprox_usd.notnull()) & (dataset_original.price_aprox_usd != 0)]

# Realizo la conversión para cada fila y calculo el promedio.

valor_dolar = dataset_dolar.apply(lambda x: x["price_aprox_local_currency"] / x["price_aprox_usd"], axis = 1).mean()

In [20]:
# Verifico que no haya ninguna fila de price aprox usd nula la cual pueda rellenar con price aprox local currency

dataset_original.loc[(dataset_original.price_aprox_local_currency.notnull()) & (dataset_original.price_aprox_usd.isnull()), ["price_aprox_local_currency", "price_aprox_usd"]]

,price_aprox_local_currency,price_aprox_usd


In [21]:
# Descarto price aprox local currency ya que, utilizare precios en USD

dataset.drop("price_aprox_local_currency", axis = 1, inplace = True)

#### Geonames id, Properati url e Image thumbnail

In [22]:
# Estas variables no aportan al fin de predecir el precio de un inmueble por lo que, las elimino.

dataset.drop("geonames_id", axis = 1, inplace = True)
dataset.drop("properati_url", axis = 1, inplace = True)
dataset.drop("image_thumbnail", axis = 1, inplace = True)

### Registros duplicados

In [23]:
# Analizo la cantidad de filas duplicadas

print("Cantidad de registros duplicados:", (dataset[dataset.duplicated()].shape)[0])

Cantidad de registros duplicados: 5086


In [24]:
# Elimino los registros duplicados

dataset.drop_duplicates(inplace = True)

### Nans por columna

In [25]:
# Analizo la cantidad de Nans por columna en todo el dataset

nulos_globales = dataset.apply(lambda x: x.isnull().sum())

### Nans por columna en porcentaje

In [26]:
# Analizo la cantidad de Nans por columna en todo el dataset a modo de porcentajes

nulos_globales.apply(lambda x: round((x * 100) / len(dataset), 2))

property_type             0.00
place_name                0.00
state_name                0.00
lat                      41.58
lon                      41.58
price_aprox_usd          15.12
surface_total_in_m2      33.05
surface_covered_in_m2    16.23
price_usd_per_m2         42.44
price_per_m2             26.21
floor                    93.41
rooms                    60.22
expenses                 87.87
description               0.00
title                     0.00
dtype: float64

### Propiedades por provincia

In [27]:
# Analizo la cantidad de provincias (state name) con su cantidad de registros

provincias = dataset.groupby("state_name")[["state_name"]].count()

# Renombro y ordeno de mayor a menor para mayor claridad

provincias = provincias.rename(columns = {"state_name" : "propiedades"})
provincias = provincias.sort_values("propiedades", ascending = False)

### Nans en price per m2 por provincia

In [28]:
# Creo un dataset con todos los registros que tengan Nan en la variable price per m2

nulos_en_per_m2 = dataset[dataset.price_per_m2.isnull()]

# Agrupo segun state name y cuento la cantidad de registros que son Nan de cada provincia

nulos_por_provincia = nulos_en_per_m2.groupby("state_name")[["state_name"]].count()

# Renombro y ordeno de mayor a menor para mayor claridad

nulos_por_provincia = nulos_por_provincia.rename(columns = {"state_name" : "registros_nulos_price_per_m2"})
nulos_por_provincia = nulos_por_provincia.sort_values("registros_nulos_price_per_m2", ascending = False)

### Visualicación en tabla con porcentajes

In [29]:
# Junto ambos dataframes anteriores

tabla_de_propiedades = pd.merge(provincias, nulos_por_provincia, on = "state_name", how = "inner")

# Calculo el porcentaje de registros nulos en price per m2 de cada provincia

tabla_de_propiedades["porcentaje_de_registros_nulos_price_per_m2"] = round((tabla_de_propiedades.registros_nulos_price_per_m2 * 100) / tabla_de_propiedades.propiedades)

tabla_de_propiedades

,propiedades,registros_nulos_price_per_m2,porcentaje_de_registros_nulos_price_per_m2
state_name,,,
Capital Federal,30745,4062,13.0
Bs.As. G.B.A. Zona Norte,24768,2651,11.0
Bs.As. G.B.A. Zona Sur,13556,3964,29.0
Córdoba,11580,5228,45.0
Buenos Aires Costa Atlántica,9914,3065,31.0
Bs.As. G.B.A. Zona Oeste,9048,2985,33.0
Santa Fe,9034,4472,50.0
Buenos Aires Interior,2251,837,37.0
Río Negro,792,584,74.0


Teniendo en cuenta la tabla, tomaré del dataset las propiedades pertenecientes a "Capital Federal" ya que, esta zona es la que más cantidad de propiedades tiene y a su vez, un bajo porcentaje de nulos.

# Análisis y limpieza CABA

In [30]:
# Creo un nuevo dataset que contenga solo las propiedades localizadas en Capital Federal

dataset_caba = dataset.loc[dataset.state_name == "Capital Federal"]

# Reinicio el índice para evitar confusiones.

dataset_caba.reset_index(inplace = True, drop = True)

In [31]:
# Creo una variable que almacene el tamaño original del dataframe para usarla a futuro

tamaño = (dataset_caba.shape)[0]
print("Tamaño del dataset:", dataset_caba.shape)

Tamaño del dataset: (30745, 15)


In [32]:
# Muestro la cantidad de nulos en cada columna

nulos = dataset_caba.apply(lambda x: x.isnull().sum())
nulos

property_type                0
place_name                   0
state_name                   0
lat                       7742
lon                       7742
price_aprox_usd           2412
surface_total_in_m2       5814
surface_covered_in_m2     2054
price_usd_per_m2          7947
price_per_m2              4062
floor                    27756
rooms                    15755
expenses                 24263
description                  0
title                        0
dtype: int64

In [33]:
# Muestro la cantidad de nulos en cada columna a modo de porcentajes

nulos.apply(lambda x: round((x * 100) / len(dataset_caba), 2))

property_type             0.00
place_name                0.00
state_name                0.00
lat                      25.18
lon                      25.18
price_aprox_usd           7.85
surface_total_in_m2      18.91
surface_covered_in_m2     6.68
price_usd_per_m2         25.85
price_per_m2             13.21
floor                    90.28
rooms                    51.24
expenses                 78.92
description               0.00
title                     0.00
dtype: float64

### Price usd per m2

Tengo propiedades que contienen su price per m2 pero, price usd per m2 esta vacío. Por lo que, mediante el valor del dólar obtenido, puedo realizar la conversión.

In [34]:
# Creo un dataset donde price usd per m2 sea nulo pero, price per m2 no lo sea.

usd_vacios = dataset_caba.loc[(dataset_caba.price_usd_per_m2.isnull()) & (dataset_caba.price_per_m2.notnull()),["price_usd_per_m2", "price_per_m2"]]

print("Cantidad de filas a convertir:", len(usd_vacios))

Cantidad de filas a convertir: 4972


In [35]:
# Realizo la conversión

usd_llenar = usd_vacios.price_per_m2.apply(lambda x: x / valor_dolar)

In [36]:
usd_llenar

30       183.420535
32       112.000367
34        91.388251
35       142.242848
36        94.192062
            ...    
30733     70.154317
30734    140.960610
30735    103.540654
30736     94.458141
30737    127.947845
Name: price_per_m2, Length: 4972, dtype: float64

In [37]:
# Aplico la serie con los valores de la conversión sobre los nans alojados en la columna price usd per m2 del dataset.

dataset_caba.loc[(dataset_caba["price_usd_per_m2"].isnull()) & (dataset_caba["price_per_m2"].notnull()), "price_usd_per_m2"] = usd_llenar

E:\Programas\anaconda3\envs\dhdsblend2021\lib\site-packages\pandas\core\indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [38]:
# Verifico que se haya realizado correctamente la imputación

print("Cantidad de filas a convertir:", (dataset_caba.loc[(dataset_caba.price_usd_per_m2.isnull()) & (dataset_caba.price_per_m2.notnull())].shape)[0])

Cantidad de filas a convertir: 0


In [39]:
# Elimino price_per_m2 ya que no será util a futuro

dataset_caba.drop("price_per_m2", axis = 1, inplace = True)

E:\Programas\anaconda3\envs\dhdsblend2021\lib\site-packages\pandas\core\frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [40]:
# Analizo nuevamente los nulos para ver como proseguir

nulos = dataset_caba.apply(lambda x: x.isnull().sum())

nulos.apply(lambda x: round((x * 100) / len(dataset_caba), 2))

property_type             0.00
place_name                0.00
state_name                0.00
lat                      25.18
lon                      25.18
price_aprox_usd           7.85
surface_total_in_m2      18.91
surface_covered_in_m2     6.68
price_usd_per_m2          9.68
floor                    90.28
rooms                    51.24
expenses                 78.92
description               0.00
title                     0.00
dtype: float64

**Objetivo: Llenar price usd per m2**

Opciones:
* Llenar price aprox usd y surface total in m2. Luego, dividir ambas columnas para obtener price usd per m2
* Llenar directamente price usd per m2

Teniendo en cuenta el porcentaje de nulos de cada variable, decidí llenar directamente price usd per m2 ya que, si desease hacer la división de las otras variables, antes tendría que imputar los valores faltantes de price usd per m2 y surface total in m2, obteniendo valores que podrían no reflejar la realidad.

### Price usd per m2

In [41]:
# Agrupo por place name y property type, las cuales son las variables que más afectan al precio, para analizar la media del precio per m2 en USD

analisis_price_usd_m2 = dataset_caba.groupby(["place_name", "property_type"])[["price_usd_per_m2"]]

# Genero estadísticas descriptivas
analisis_price_usd_m2.describe()

price_usd_per_m2                            \
                                          count         mean          std   
place_name       property_type                                              
Abasto           PH                        13.0  1570.485350   587.602861   
                 apartment                 73.0  1823.241579   912.143314   
                 house                      3.0  1432.703521   196.391518   
                 store                      7.0  2080.293122   506.695161   
Agronomía        PH                         7.0   847.681223   756.022818   
...                                         ...          ...          ...   
Villa Urquiza    store                     19.0  1864.938986  1769.069086   
Villa del Parque PH                        46.0  1558.496136  1956.686036   
                 apartment                318.0  1902.892248   989.122756   
                 house                     46.0  1026.536106   697.444457   
                 store                     13.0  1877.990794  2135.548694   

                                                                       \
                                        min          25%          50%   
place_name       property_type                                          
Abasto           PH              864.406780  1000.000000  1500.000000   
                 apartment        88.984865  1366.666667  2133.333333   
                 house          1225.000000  1341.362974  1457.725948   
                 store          1428.571429  1880.757503  1900.000000   
Agronomía        PH              101.048244   127.467641   878.460711   
...                                     ...          ...          ...   
Villa Urquiza    store           141.687211   427.895378  1461.538462   
Villa del Parque PH               96.967185   425.061634  1383.389075   
                 apartment        63.291749  1738.461538  2170.507042   
                 house            51.746634   143.492119  1105.125035   
                 store            90.964959   935.135595  1400.000000   

                                                           
                                        75%           max  
place_name       property_type                             
Abasto           PH             2000.000000   2500.000000  
                 apartment      2511.259259   2939.393939  
                 house          1536.555281   1615.384615  
                 store          2315.528163   2840.909091  
Agronomía        PH             1349.662162   2000.000000  
...                                     ...           ...  
Villa Urquiza    store          2658.208644   7269.778123  
Villa del Parque PH             1790.710888  12185.100181  
                 apartment      2585.937500   5610.813572  
                 house          1517.994679   2543.103448  
                 store          1894.736842   8300.000000  

[235 rows x 8 columns]

Teniendo en cuenta que, para algunos casos, el desvío estándar es elevado y que, la media difiere mucho del percentil 75 y del máximo. Para imputar valores, utilizare la media recortada excluyendo el 25% de cada extremo, es decir, la media del rango intercuartil.

In [42]:
print("Cantidad de filas a imputar:", (dataset_caba[dataset_caba.price_usd_per_m2.isnull()].shape)[0])

Cantidad de filas a imputar: 2975


**stats.trim_mean** es una función que me permite calcular la media recortada, enviadole como parámetro sobre que quiero calcular la media (x) y que porcentaje de los extremos deseo excluir (0.25 = 25%).

In [43]:
# Mediante un transform sobre los grupos, calculo la media recortada del precio per m2 en USD

media_price_usd_m2 = analisis_price_usd_m2.transform(lambda x: stats.trim_mean(x,0.25))

In [44]:
# Relleno cada valor nan de price usd per m2 con su valor correspondiente de media price usd m2

dataset_caba.loc[dataset_caba.price_usd_per_m2.isnull(), "price_usd_per_m2"] = media_price_usd_m2[dataset_caba.price_usd_per_m2.isnull()]

E:\Programas\anaconda3\envs\dhdsblend2021\lib\site-packages\pandas\core\indexing.py:1715: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


In [45]:
print("Cantidad de filas a imputar:", (dataset_caba[dataset_caba.price_usd_per_m2.isnull()].shape)[0])

Cantidad de filas a imputar: 177


Para imputar los registros faltantes, utilizare la división entre price aprox usd y surface total in m2

### Price aprox usd

Primero, rellenaré valores nulos mediante la multiplicación entre la variable price usd per m2 y la variable surface total in m2. Luego, en caso de seguir teniendo valores nulos, los rellenare mediante cálculos estadísticos.

Los primeros valores imputados no me servirán para llenar price usd per m2 ya que, en estos casos el valor de price usd per m2 no es nulo (lo utilizo para hacer la multiplicación). Esto lo realizo para que, los valores de price aprox usd sean lo más reales posibles

In [46]:
print("Cantidad de filas a imputar:", (dataset_caba[dataset_caba.price_aprox_usd.isnull()].shape)[0])

Cantidad de filas a imputar: 2412


In [47]:
# Analizo que filas puedo completar mediante la multiplicación entre price usd per m2 y surface total in m2

fill_producto = dataset_caba.loc[(dataset_caba.price_usd_per_m2.notnull()) & (dataset_caba.surface_total_in_m2.notnull()) & (dataset_caba.price_aprox_usd.isnull())]

In [48]:
# Lleno los nulos de price aprox usd con el resultado del producto entre price usd per m2 y surface total in m2

fill_producto.price_aprox_usd = fill_producto.price_usd_per_m2 * fill_producto.surface_total_in_m2

E:\Programas\anaconda3\envs\dhdsblend2021\lib\site-packages\pandas\core\generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [49]:
# Imputo los valores obtenidos al dataset

dataset_caba.iloc[fill_producto.index] = fill_producto

In [50]:
print("Cantidad de filas a imputar:", (dataset_caba[dataset_caba.price_aprox_usd.isnull()].shape)[0])

Cantidad de filas a imputar: 368


Estos registros faltantes, los rellenaré mediante cálculos estadísticos

In [51]:
# Agrupo por place name y property type, las cuales son las variables que más afectan al precio, para analizar la media del precio en USD

analisis_price_usd = dataset_caba.groupby(["place_name", "property_type"])[["price_aprox_usd"]]

# Genero estadisticas descriptivas 

analisis_price_usd.describe()


price_aprox_usd                                \
                                         count           mean            std   
place_name       property_type                                                 
Abasto           PH                       14.0  173537.346317  115214.555200   
                 apartment                75.0  127697.298094   60572.734314   
                 house                     3.0  470000.000000   43588.989435   
                 store                     9.0  371476.949235  324410.080866   
Agronomía        PH                        7.0  210714.285714   58696.150600   
...                                        ...            ...            ...   
Villa Urquiza    store                    24.0  387462.101526  830032.150168   
Villa del Parque PH                       46.0  174156.521739   72833.992834   
                 apartment               345.0  149943.427576   75364.103155   
                 house                    49.0  364891.856872  126682.587009   
                 store                    13.0  377798.260000  237069.244239   

                                                                          \
                                          min         25%            50%   
place_name       property_type                                             
Abasto           PH              55000.000000  106275.000  160000.000000   
                 apartment       38000.000000   86357.435  130000.000000   
                 house          420000.000000  455000.000  490000.000000   
                 store           38292.543114  160000.000  380000.000000   
Agronomía        PH             130000.000000  175000.000  210000.000000   
...                                       ...         ...            ...   
Villa Urquiza    store           48000.000000   89750.000  158450.000000   
Villa del Parque PH              79900.000000  113500.000  161000.000000   
                 apartment       59459.000000   95000.000  135267.174629   
                 house          164000.000000  275000.000  355000.000000   
                 store           22377.380000  184000.000  330000.000000   

                                                             
                                          75%           max  
place_name       property_type                               
Abasto           PH             195750.000000  5.100000e+05  
                 apartment      150116.000000  4.400000e+05  
                 house          495000.000000  5.000000e+05  
                 store          380000.000000  1.160000e+06  
Agronomía        PH             237500.000000  3.100000e+05  
...                                       ...           ...  
Villa Urquiza    store          311569.601258  4.199901e+06  
Villa del Parque PH             225250.000000  3.300000e+05  
                 apartment      185600.000000  6.550000e+05  
                 house          450000.000000  6.500000e+05  
                 store          630000.000000  8.300000e+05  

[235 rows x 8 columns]

Debido a los altos valores en el desvío estándar y a la diferencia entre la media, el percentil 25, 75 y el máximo, para mis cálculos utilizaré la media recortada

In [52]:
# Mediante un transform sobre los grupos, calculo la media recortada del precio en USD

media_price_usd = analisis_price_usd.transform(lambda x: stats.trim_mean(x,0.25))

In [53]:
# Relleno cada valor nan de price aprox usd con su valor correspondiente de media price usd

dataset_caba.loc[dataset_caba.price_aprox_usd.isnull(), "price_aprox_usd"] = media_price_usd[dataset_caba.price_aprox_usd.isnull()]

E:\Programas\anaconda3\envs\dhdsblend2021\lib\site-packages\pandas\core\indexing.py:1715: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


In [54]:
print("Cantidad de filas a imputar:", (dataset_caba[dataset_caba.price_aprox_usd.isnull()].shape)[0])

Cantidad de filas a imputar: 65


### Surface total in m2

Primero llenaré valores nulos de surface total in m2 mediante la división entre price aprox usd y price usd per m2. Luego, en caso de tener registros nulos, los completare con cálculos estadísticos

In [55]:
print("Cantidad de filas a imputar:", (dataset_caba.loc[dataset_caba.surface_total_in_m2.isnull()].shape)[0])

Cantidad de filas a imputar: 5814


In [56]:
# Creo un dataset con las filas que puedo completar mediante la división entre price aprox usd y price usd per m2

llenar_surface_total = dataset_caba.loc[(dataset_caba.surface_total_in_m2.isnull()) & (dataset_caba.price_aprox_usd.notnull()) & (dataset_caba.price_usd_per_m2.notnull())]

In [57]:
# Completo los valores nulos de surface total in m2 con los resultados de la división para cada fila

llenar_surface_total.surface_total_in_m2 = llenar_surface_total.price_aprox_usd / llenar_surface_total.price_usd_per_m2

E:\Programas\anaconda3\envs\dhdsblend2021\lib\site-packages\pandas\core\generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [58]:
# Imputo en las celdas nan de surface total in m2 del dataset, los valores obtenidos

dataset_caba.iloc[llenar_surface_total.index] = llenar_surface_total

In [59]:
print("Cantidad de filas a imputar:", (dataset_caba.loc[dataset_caba.surface_total_in_m2.isnull()].shape)[0])

Cantidad de filas a imputar: 101


Estos registros faltantes los completaré mediante cálculos estadísticos

In [60]:
# Agrupo por place name y property type, las cuales son las variables que más afectan a la superficie, para analizar la media de la superficie total

analisis_surface_total = dataset_caba.groupby(["place_name", "property_type"])[["surface_total_in_m2"]]

# Genero estadisticas descriptivas 

analisis_surface_total.describe()


surface_total_in_m2                            \
                                             count         mean          std   
place_name       property_type                                                 
Abasto           PH                           14.0   136.000000   140.518819   
                 apartment                    76.0   224.585412   397.083217   
                 house                         5.0   230.600000   150.513455   
                 store                         9.0   187.508918   175.995881   
Agronomía        PH                            7.0   874.586714   903.652058   
...                                            ...          ...          ...   
Villa Urquiza    store                        24.0   293.132494   430.022291   
Villa del Parque PH                           46.0   405.075924   590.343620   
                 apartment                   352.0   244.453141   434.311390   
                 house                        50.0  1148.699528  1635.392581   
                 store                        13.0   501.129231   819.514236   

                                                                            \
                                      min     25%         50%          75%   
place_name       property_type                                               
Abasto           PH             22.000000   66.25  103.000000   155.250000   
                 apartment      24.000000   42.25   58.000000   104.000000   
                 house          75.000000   75.00  260.000000   343.000000   
                 store          17.000000  112.00  177.580266   200.000000   
Agronomía        PH             65.000000  152.50  352.890000  1561.538249   
...                                   ...     ...         ...          ...   
Villa Urquiza    store          19.791367   74.25  168.000000   352.890000   
Villa del Parque PH             17.644500   78.50  129.500000   352.890000   
                 apartment      17.644500   49.00   65.500000   101.000000   
                 house          78.000000  222.00  352.890000  1276.070249   
                 store          38.000000  175.00  352.890000   450.000000   

                                             
                                        max  
place_name       property_type               
Abasto           PH              590.000000  
                 apartment      1887.961499  
                 house           400.000000  
                 store           620.000000  
Agronomía        PH             2276.140499  
...                                     ...  
Villa Urquiza    store          1905.000000  
Villa del Parque PH             2646.674998  
                 apartment      3475.966498  
                 house          5857.973996  
                 store          3176.009998  

[235 rows x 8 columns]

Debido a los altos valores en el desvío estándar y a la diferencia entre la media, el percentil 25, 75 y el máximo, para mis cálculos utilizaré la media recortada

In [61]:
# Mediante un transform sobre los grupos, calculo la media recortada de la superficie total

media_surface_total = analisis_surface_total.transform(lambda x: stats.trim_mean(x,0.25))

In [62]:
# Relleno cada valor nan de surface total in m2 con su valor correspondiente de media surface total

dataset_caba.loc[dataset_caba.surface_total_in_m2.isnull(), "surface_total_in_m2"] = media_surface_total[dataset_caba.surface_total_in_m2.isnull()]

E:\Programas\anaconda3\envs\dhdsblend2021\lib\site-packages\pandas\core\indexing.py:1715: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


In [63]:
print("Cantidad de filas a imputar:", (dataset_caba.loc[dataset_caba.surface_total_in_m2.isnull()].shape)[0])

Cantidad de filas a imputar: 75


### Price usd per m2

En price usd per m2, aun me quedan algunos registros sin completar los cuales rellenaré mediante la división entre price aprox usd y surface total in m2

In [64]:
print("Cantidad de filas a imputar:", (dataset_caba[dataset_caba.price_usd_per_m2.isnull()].shape)[0])

Cantidad de filas a imputar: 177


In [65]:
# Selecciono solo las filas que contienen la información para rellenar

fill_division = dataset_caba.loc[(dataset_caba.price_usd_per_m2.isnull()) & (dataset_caba.price_aprox_usd.notnull()) & (dataset_caba.surface_total_in_m2.notnull())]

In [66]:
# LLeno los valores nulos de price usd per m2 mediante la división de las variables correspondientes

fill_division.price_usd_per_m2 = fill_division.price_aprox_usd / fill_division.surface_total_in_m2

E:\Programas\anaconda3\envs\dhdsblend2021\lib\site-packages\pandas\core\generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [67]:
# Imputo en el dataset los valores faltantes de price usd per m2

dataset_caba.iloc[fill_division.index] = fill_division

In [68]:
print("Cantidad de filas a imputar:", (dataset_caba[dataset_caba.price_usd_per_m2.isnull()].shape)[0])

Cantidad de filas a imputar: 138


Los registros que me han quedado en nulo, son aquellos en los cuales no se pudo realizar la división porque price aprox usd es nulo o surface total in m2 es nulo.

Los nulos de **price aprox usd** y **surface total in m2** surgen ya que, originalmente, a la hora de completar los nulos de estas variables con la media recortada, los valores extremos (fuera de los percentiles 25 y 75) quedaban sin una media especificada.

Para no tener estos registros nulos, debería de hacer mis cálculos con la media. Sin embargo, prefiero eliminar estos registros a fin de tener unos valores más cercanos a la realidad a costa de perder dispersión en los datos.

**Aclaración: La eliminación de las filas con Nans se hará al final de la notebook**

### Surface covered in m2

In [69]:
print("Cantidad de filas a imputar:", (dataset_caba[dataset_caba.surface_covered_in_m2.isnull()].shape)[0])

Cantidad de filas a imputar: 2054


Para completar los valores nulos de surface covered, utilizare cálculos estadísticos

In [70]:
# Agrupo por place name y property type, las cuales son las variables que mas afectan a la superficie, para analizar la media de la superficie cubierta

analisis_surface_covered = dataset_caba.groupby(["place_name", "property_type"])[["surface_covered_in_m2"]]

# Genero estadisticas descriptivas 

analisis_surface_covered.describe()


surface_covered_in_m2                          \
                                               count        mean         std   
place_name       property_type                                                 
Abasto           PH                             13.0  102.230769  116.676300   
                 apartment                      71.0   54.816901   26.004016   
                 house                           4.0  137.500000  153.215534   
                 store                           7.0  177.285714  203.113855   
Agronomía        PH                              7.0   94.714286   51.622716   
...                                              ...         ...         ...   
Villa Urquiza    store                          22.0  153.863636  397.152279   
Villa del Parque PH                             46.0   72.978261   42.349073   
                 apartment                     341.0   58.829912   34.457226   
                 house                          47.0  177.212766  112.689635   
                 store                          12.0  188.916667  179.628636   

                                                                   
                                 min    25%    50%    75%     max  
place_name       property_type                                     
Abasto           PH             22.0  47.00   68.0   94.0   470.0  
                 apartment      24.0  37.00   50.0   61.5   164.0  
                 house           5.0   5.00  132.5  265.0   280.0  
                 store          17.0  78.00  118.0  165.0   620.0  
Agronomía        PH             20.0  74.00   87.0  111.5   185.0  
...                              ...    ...    ...    ...     ...  
Villa Urquiza    store          20.0  20.00   35.5  102.0  1905.0  
Villa del Parque PH              1.0  45.25   65.0   90.0   180.0  
                 apartment       1.0  42.00   58.0   63.0   481.0  
                 house           1.0  99.50  175.0  232.0   410.0  
                 store          20.0  33.50  140.0  297.0   481.0  

[235 rows x 8 columns]

Debido a los altos valores en el desvío estándar y a la diferencia entre la media, el percentil 25, 75 y el máximo, para mis cálculos utilizaré la media recortada

In [71]:
# Mediante un transform sobre los grupos, calculo la media recortada de la superficie cubierta

media_surface_covered = analisis_surface_covered.transform(lambda x: stats.trim_mean(x,0.25))

In [72]:
# Relleno cada valor nan de surface covered in m2 con su valor correspondiente de media surface covered

dataset_caba.loc[dataset_caba.surface_covered_in_m2.isnull(), "surface_covered_in_m2"] = media_surface_covered[dataset_caba.surface_covered_in_m2.isnull()]

E:\Programas\anaconda3\envs\dhdsblend2021\lib\site-packages\pandas\core\indexing.py:1715: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


In [73]:
# Verifico que no me hayan quedado registros en nulo.

print("Cantidad de filas a imputar:", (dataset_caba.loc[dataset_caba.surface_covered_in_m2.isnull()].shape)[0])

Cantidad de filas a imputar: 299


**Estos registros seran eliminados al final de la notebook**


### Lat

A fin de rellenar los valores nulos en la variable lat, agruparé las propiedades según su ubicación y hare un promedio de la latitud. Por lo que, obtendré un latitud aproximada para cada barrio

In [74]:
print("Cantidad de filas a imputar:", (dataset_caba.loc[dataset_caba.lat.isnull()].shape)[0])

Cantidad de filas a imputar: 7742


In [75]:
# Agrupo las propiedades según su place name y sobre la latitud calculo la media

lat_aprox = dataset_caba.groupby("place_name")[["lat"]].mean()

# Renombro para que al realizar el merge no me modifique el nombre de la variable original en el dataset
lat_aprox.rename(columns = {"lat": "prom_latitud"}, inplace = True)


In [76]:
# Agrego al dataset, el promedio de la latitud correspondiente a cada propiedad según su place name

dataset_caba = pd.merge(lat_aprox, dataset_caba, on = "place_name", how = "inner")

In [77]:
# Relleno los valores nulos de lat con los valores brindados por el promedio de la latitud según su place name

dataset_caba.lat.fillna(dataset_caba.prom_latitud, axis = 0, inplace = True)

In [78]:
# Elimino la columna que contiene el promedio de la latitud ya que no será útil a futuro

dataset_caba.drop("prom_latitud", axis = 1, inplace = True)

In [79]:
# Verifico que se haya realizado correctamente la imputación

print("Cantidad de filas a imputar:", (dataset_caba.loc[dataset_caba.lat.isnull()].shape)[0])

Cantidad de filas a imputar: 0


### Lon

Para llenar la variable Lon, voy a seguir la misma estrategia utilizada para la variable Lat

In [80]:
print("Cantidad de filas a imputar:", (dataset_caba.loc[dataset_caba.lon.isnull()].shape)[0])

Cantidad de filas a imputar: 7742


In [81]:
# Agrupo las propiedades según su place name y sobre la longitud calculo la media

lon_aprox = dataset_caba.groupby("place_name")[["lon"]].mean()

# Renombro para que al realizar el merge no me modifique el nombre de la variable original en el dataset
lon_aprox.rename(columns = {"lon": "prom_longitud"}, inplace = True)


In [82]:
# Agrego al dataset, el promedio de la longitud correspondiente a cada propiedad según su place name

dataset_caba = pd.merge(lon_aprox, dataset_caba, on = "place_name", how = "inner")

In [83]:
# Relleno los valores nulos de lon con los valores brindados por el promedio de la longitud según su place name

dataset_caba.lon.fillna(dataset_caba.prom_longitud, axis = 0, inplace = True)

In [84]:
# Elimino la columna que contiene el promedio de la longitud ya que no será útil a futuro

dataset_caba.drop("prom_longitud", axis = 1, inplace = True)

In [85]:
# Verifico que se haya realizado correctamente la imputación

print("Cantidad de filas a imputar:", (dataset_caba.loc[dataset_caba.lon.isnull()].shape)[0])

Cantidad de filas a imputar: 0


### Garage

In [86]:
# Busco en la descripción y en el título aquellas propiedades que contengan la palabra garage o similares

garage = dataset_caba.description.apply(lambda x: re.search("garage|estacionamiento|cochera|entrada para auto|entrada de auto|entrada de coche|entrada para coche", str(x), flags = re.I))
garage_titulo = dataset_caba.title.apply(lambda x: re.search("garage|estacionamiento|cochera|entrada para auto|entrada de auto|entrada de coche|entrada para coche", str(x), flags = re.I))

# Guardo en una nueva variable solo los objetos matchs

garage_matchs = garage[garage.notnull()]
garage_titulo_matchs = garage_titulo[garage_titulo.notnull()]


In [87]:
garage_matchs

14       <re.Match object; span=(46, 61), match='ESTACI...
16       <re.Match object; span=(679, 686), match='coch...
31       <re.Match object; span=(583, 590), match='COCH...
38       <re.Match object; span=(672, 687), match='Esta...
40       <re.Match object; span=(830, 837), match='Coch...
                               ...                        
30734    <re.Match object; span=(162, 169), match='coch...
30735    <re.Match object; span=(395, 402), match='COCH...
30736    <re.Match object; span=(293, 300), match='Coch...
30737    <re.Match object; span=(430, 437), match='COCH...
30738    <re.Match object; span=(82, 89), match='Cochera'>
Name: description, Length: 11253, dtype: object

In [88]:
# Agrego una columna al dataset llamada garage con el valor por defecto "No"

dataset_caba["garage"] = 0

In [89]:
# Modifico a "Si" el valor de la variable garage para las propiedades que correspondan

dataset_caba.garage.iloc[garage_matchs.index] = 1
dataset_caba.garage.iloc[garage_titulo_matchs.index] = 1


E:\Programas\anaconda3\envs\dhdsblend2021\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [90]:
# Verifico que no me haya quedado alguna propiedad en nulo

print("Cantidad de propiedades a imputar:", (dataset_caba.loc[dataset_caba.garage.isnull()].shape)[0])

Cantidad de propiedades a imputar: 0


### Ambientes

In [91]:
print("Cantidad de propiedades a imputar:", (dataset_caba.loc[dataset_caba.rooms.isnull()].shape)[0])

Cantidad de propiedades a imputar: 15755


In [92]:
# Busco en la descripción y en el título aquellas propiedades que contengan la palabra ambiente o similares y traigo los números cercanos a izquierda

ambientes = dataset_caba.description.apply(lambda x: re.search("\d+\s*(amb|ambientes|habitaciones|dormitorios)", str(x), flags = re.I))
ambientes_titulo = dataset_caba.title.apply(lambda x: re.search("\d+\s*(amb|ambientes|habitaciones|dormitorios)", str(x), flags = re.I))

# Guardo en una nueva variable solo los objetos matchs

ambientes = ambientes[ambientes.notnull()]
ambientes_titulo = ambientes_titulo[ambientes_titulo.notnull()]

In [93]:
# Busco en la descripcion y en el título aquellas propiedades que contengan la palabra monoambiente

monoambiente = dataset_caba.description.apply(lambda x: re.search("monoambiente", str(x), flags = re.I))
monoambiente_titulo = dataset_caba.title.apply(lambda x: re.search("monoambiente", str(x), flags = re.I))

# Guardo en una nueva variable solo los objetos matchs

monoambiente = monoambiente[monoambiente.notnull()]
monoambiente_titulo = monoambiente_titulo[monoambiente_titulo.notnull()]

In [94]:
# Guardo en la variable solo el resultado encontrado

ambientes = ambientes.apply(lambda x: x.group())
ambientes_titulo = ambientes_titulo.apply(lambda x: x.group())

In [95]:
# Cambio todas las letras que estén en mayúscula a minúscula

ambientes = ambientes.apply(lambda x: x.lower())
ambientes_titulo = ambientes_titulo.apply(lambda x: x.lower())

In [96]:
# Elimino todas las apariciones de "amb" y "habitaciones" para quedarme solo con el número de ambientes
                                          
ambientes = ambientes.apply(lambda x: x.rstrip("amb"))
ambientes = ambientes.apply(lambda x: x.rstrip("habitaciones"))
ambientes = ambientes.apply(lambda x: x.rstrip("dormitorios"))

ambientes_titulo = ambientes_titulo.apply(lambda x: x.rstrip("amb"))
ambientes_titulo = ambientes_titulo.apply(lambda x: x.rstrip("habitaciones"))
ambientes_titulo = ambientes_titulo.apply(lambda x: x.rstrip("dormitorios"))


In [97]:
# Me quedo solo con las propiedades las cuales en el dataset original tiene como valor de room "nan" ya que, no quiero modificar las propiedades que ya contienen un valor.

ambientes = ambientes[dataset_caba.rooms.isnull()]
ambientes_titulo = ambientes_titulo[dataset_caba.rooms.isnull()]

monoambiente = monoambiente[dataset_caba.rooms.isnull()]
monoambiente_titulo = monoambiente_titulo[dataset_caba.rooms.isnull()]

In [98]:
# Debido a que el match me trae objetos del tipo str, los convierto a int

ambientes = pd.to_numeric(ambientes)
ambientes_titulo = pd.to_numeric(ambientes_titulo)

Primero, voy a imputar los valores encontrados en la descripción y luego los valores encontrados en el título. Esto lo realizo de esta manera ya que, me parecen más fieles los datos encontrados en el título y, de esta manera, si para una propiedad encontré un numero de ambientes en el título y otro en la descripción, el número final que será imputado es el del título

In [99]:
# En las propiedades que contengan el campo rooms en nulo, inserto el numero de ambientes encontrado en la descripción

dataset_caba.rooms.iloc[monoambiente.index] = 1
dataset_caba.rooms.iloc[ambientes.index] = ambientes

In [100]:
# En las propiedades que contengan el campo rooms en nulo, inserto el numero de ambientes encontrado en el título

dataset_caba.rooms.iloc[monoambiente_titulo.index] = 1
dataset_caba.rooms.iloc[ambientes_titulo.index] = ambientes_titulo

In [101]:
# Verifico si aún tengo registros con la variable rooms en nulo

print("Cantidad de propiedades a imputar:", (dataset_caba.loc[dataset_caba.rooms.isnull()].shape)[0])

Cantidad de propiedades a imputar: 5129


Para las variables nulas de room restantes, voy a imputar los valores mediante calculos estadísticos

In [102]:
# Calculo la media para cada propiedad según su place name y property type.

media_room = dataset_caba.groupby(["place_name", "property_type"])[["rooms"]].mean()

# Renombro para que a la hora de hacer el merge no me altere las columnas

media_room.rename(columns = {"rooms" : "media_room"}, inplace = True)

In [103]:
# Debido a que necesito valores enteros por la naturaleza de la variable. Además, redondeo cada dato hacia abajo tambien por la naturaleza de la variable

media_room = media_room.apply(lambda x: np.floor(x))

In [104]:
# Agrego al dataset, el valor correspondiente de ambientes segun el place name y property type de la propiedad

dataset_caba = pd.merge(media_room, dataset_caba, on = ["place_name", "property_type"], how = "inner")

In [105]:
# Relleno los valores nulos de rooms con los valores brindados por el promedio de rooms según su place name y property type

dataset_caba.rooms.fillna(dataset_caba.media_room, axis = 0, inplace = True)

In [106]:
# Elimino la columna media room

dataset_caba.drop("media_room", axis = 1, inplace = True)

In [107]:
# Verifico si aún tengo registros con la variable rooms en nulo

print("Cantidad de propiedades a imputar:", (dataset_caba.loc[dataset_caba.rooms.isnull()].shape)[0])

Cantidad de propiedades a imputar: 180


**Estos registros seran eliminados al final de la notebook**


### Amb al exterior

En esta variable, se colocará un "Si" en caso de que la propiedad tenga algún ambiente que este en el exterior ya sea, un jardín, patio o terraza.

In [108]:
# Busco en la descripción y en el título aquellas propiedades que contengan la palabra jardin, patio o terraza

amb_al_exterior = dataset_caba.description.apply(lambda x: re.search("jardin|jardín|patio|terraza", str(x), flags = re.I))
amb_al_exterior_titulo = dataset_caba.title.apply(lambda x: re.search("jardin|jardín|patio|terraza", str(x), flags = re.I))

# Guardo en una nueva variable solo los objetos matchs

amb_al_exterior_matchs = amb_al_exterior[amb_al_exterior.notnull()]
amb_al_exterior_titulo_matchs = amb_al_exterior_titulo[amb_al_exterior_titulo.notnull()]

In [109]:
# Agrego una columna al dataset llamada amb al exterior con el valor por defecto "No"

dataset_caba["amb_al_exterior"] = 0

In [110]:
# Modifico a "Si" el valor de la variable amb al exterior para las propiedades que correspondan

dataset_caba.amb_al_exterior.iloc[amb_al_exterior_matchs.index] = 1
dataset_caba.amb_al_exterior.iloc[amb_al_exterior_titulo_matchs.index] = 1

In [111]:
# Verifico que no me haya quedado alguna propiedad en nulo

print("Cantidad de propiedades a imputar:", (dataset_caba.loc[dataset_caba.amb_al_exterior.isnull()].shape)[0])

Cantidad de propiedades a imputar: 0


### Balcón

In [112]:
# Busco en la descripción y en el título aquellas propiedades que contengan la palabra balcón

balcon = dataset_caba.description.apply(lambda x: re.search("balcon|balcón", str(x), flags = re.I))
balcon_titulo = dataset_caba.title.apply(lambda x: re.search("balcon|balcón", str(x), flags = re.I))

# Guardo en una nueva variable solo los objetos matchs

balcon_matchs = balcon[balcon.notnull()]
balcon_titulo_matchs = balcon_titulo[balcon_titulo.notnull()]

In [113]:
balcon_matchs

0        <re.Match object; span=(237, 243), match='BALC...
1        <re.Match object; span=(237, 243), match='BALC...
3        <re.Match object; span=(404, 410), match='balc...
7        <re.Match object; span=(224, 230), match='balc...
15       <re.Match object; span=(237, 243), match='BALC...
                               ...                        
30720    <re.Match object; span=(894, 900), match='balc...
30727    <re.Match object; span=(803, 809), match='balc...
30730    <re.Match object; span=(1144, 1150), match='ba...
30731    <re.Match object; span=(688, 694), match='balc...
30735    <re.Match object; span=(392, 398), match='BALC...
Name: description, Length: 14451, dtype: object

In [114]:
dataset_caba

,place_name,property_type,state_name,lat,lon,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,floor,rooms,expenses,description,title,garage,amb_al_exterior
0,Abasto,PH,Capital Federal,-34.606569,-58.411137,160000.000000,160.000000,63.0,1000.000000,1.0,4.0,NaN,"4 AMBIENTES CON PATIO Y TERRAZA AL FRENTE, EN ...",PH EN VENTA,0,1
1,Abasto,PH,Capital Federal,-34.606569,-58.411137,160000.000000,160.000000,63.0,1000.000000,1.0,4.0,NaN,"4 AMBIENTES CON PATIO Y TERRAZA AL FRENTE, EN ...",PH EN VENTA,0,1
2,Abasto,PH,Capital Federal,-34.601053,-58.408352,136922.848443,86.000000,75.0,1592.126145,NaN,4.0,NaN,"Venta de Casa 4 AMBIENTES en Abasto, Capital F...",PH EN VENTA,0,1
3,Abasto,PH,Capital Federal,-34.606465,-58.411705,254900.000000,200.000000,161.0,1274.500000,NaN,5.0,NaN,Sarmiento 3200. PH en Abasto reciclado a nuevo...,Impecable Ph reciclado en Abasto,0,1
4,Abasto,PH,Capital Federal,-34.603715,-58.410990,67500.000000,45.000000,38.0,1500.000000,NaN,2.0,NaN,Departamento Tipo Casa en Venta 2 Ambientes y ...,Departamento Tipo Casa en Venta 2 Dormitorios,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30740,Villa del Parque,store,Capital Federal,-34.614552,-58.479048,315000.000000,3176.009998,180.0,99.181048,NaN,5.0,1100.0,LOCAL DE 180 MTS CUBIERTOS MAS 45 DESCUBIERTOS...,Local - Villa del Parque,0,1
30741,Villa del Parque,store,Capital Federal,-34.607255,-58.488449,830000.000000,100.000000,100.0,8300.000000,NaN,5.0,NaN,Venta de Local en Villa del ParqueVenta de lo...,Local en Villa del Parque,0,0
30742,Villa del Parque,store,Capital Federal,-34.612562,-58.480630,630000.000000,450.000000,450.0,1400.000000,NaN,5.0,NaN,Venta de Local en Villa del ParqueLOCAL EN 3 ...,Local en Villa del Parque,0,0
30743,Villa del Parque,store,Capital Federal,-34.612562,-58.480630,630000.000000,450.000000,450.0,1400.000000,NaN,5.0,NaN,Venta de Local en Villa del ParqueLOCAL EN 3 ...,Local - Villa del Parque,0,0


In [115]:
# Agrego una columna al dataset llamada balcon con el valor por defecto "No"

dataset_caba["balcon"] = 0

In [116]:
# Modifico a "Si" el valor de la variable balcon para las propiedades que correspondan

dataset_caba.balcon.iloc[balcon_matchs.index] = 1
dataset_caba.balcon.iloc[balcon_titulo_matchs.index] = 1

E:\Programas\anaconda3\envs\dhdsblend2021\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [117]:
# Verifico que no me haya quedado alguna propiedad en nulo

print("Cantidad de propiedades a imputar:", (dataset_caba.loc[dataset_caba.amb_al_exterior.isnull()].shape)[0])

Cantidad de propiedades a imputar: 0


### Expenses y Floor

Para rellenar nans en estas variables, lo adecuado sería utilizar expresiones regulares sobre la descripción y el título. Sin embargo, debido a la amplia cantidad de formas en las que el usuario puede escribir acerca de estas variables. Decidí eliminarlas del dataset.

In [118]:
# Elimino las columnas expenses y floor

dataset_caba.drop("expenses", axis = 1, inplace = True)
dataset_caba.drop("floor", axis = 1, inplace = True)

### Registros nulos

Los registros que han quedado con nulo en alguna de sus columnas serán eliminados.

In [119]:
cant_nulos = len(dataset_caba) - len(dataset_caba.dropna())
print("Cantidad de registros nulos:", cant_nulos)

Cantidad de registros nulos: 521


In [120]:
dataset_caba.dropna(inplace = True)

### Resultados finales

In [121]:
print("El dataset reducido a 'Capital Federal', comenzó con una cantidad inicial de", tamaño, "registros. Se logró imputar todos los valores faltantes eliminando", cant_nulos, "registros.")

El dataset reducido a 'Capital Federal', comenzó con una cantidad inicial de 30745 registros. Se logró imputar todos los valores faltantes eliminando 521 registros.


In [122]:
print("Cantidad de nulos:")
print()
print(dataset_caba.apply(lambda x: x.isnull().sum()))


Cantidad de nulos:

place_name               0
property_type            0
state_name               0
lat                      0
lon                      0
price_aprox_usd          0
surface_total_in_m2      0
surface_covered_in_m2    0
price_usd_per_m2         0
rooms                    0
description              0
title                    0
garage                   0
amb_al_exterior          0
balcon                   0
dtype: int64


In [123]:
# Exporto el dataset a csv

dataset_caba.to_csv("dataset_caba.csv")

In [272]:
data = dataset_caba

In [274]:
data = data[data.price_usd_per_m2 < 30000]

In [275]:
df_dummies = data

In [276]:
df_dummies = pd.concat([data, pd.get_dummies(data = data['place_name'], prefix= 'place_name_')], axis = 1)

In [277]:
df_dummies = pd.concat([df_dummies, pd.get_dummies(data = data['property_type'], prefix= 'property_type_')], axis = 1)

In [278]:
df_dummies = df_dummies.drop(["price_aprox_usd", "surface_total_in_m2", "surface_covered_in_m2", "place_name", "property_type", "state_name", "lat", "lon", "description", "title"], axis = 1)

In [279]:
normalizar = ['rooms']
transformer = StandardScaler().fit(df_dummies[normalizar])

std_df = pd.DataFrame(transformer.transform(df_dummies[normalizar]))
std_df.columns = [i + '_std' for i in normalizar]
std_df

,rooms_std
0,0.014313
1,0.014313
2,0.014313
3,0.034917
4,-0.026896
...,...
6784,-0.047500
6785,-0.047500
6786,-0.026896
6787,-0.047500
